# Naive Bayes (classificador de spam)

Esse exemplo foi adaptado de Frank Kane da Sundog Education http://sundog-education.com/ e qualquer reprodução deve referenciar seus produtores

#Etapa 1: realizar a leitura dos e-mails e colocá-los em um dataframe

In [9]:
import os #biblioteca para acessar funções do S.O.
import io #biblioteca para manipulação de arquivos
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

#função de leitura do arquivo de e-mails 
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            # encontra o arquivo
            path = os.path.join(root, filename)
            inBody = False
            lines = []
            # abre o arquivo
            f = io.open(path, 'r', encoding='latin1')
            # le p arquivo até o final separando por linhas
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            #retorna os valores
            yield path, message


Etapa 2: Criando os DataFrames para a manipulação dos dados. Nossos DataFrames possuem dois campos: mensagem e o rótulo atribuido a priori

In [24]:
import numpy as np
from pandas import DataFrame

#cria uma estrutura de dados à partir dos dados coletados do arquivo
def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)
    return DataFrame(rows, index=index)

# coloque aqui o caminho do diretório onde se encontram os e-mails na sua máquina
path = "/var/www/html/pos-graduacao/RDP/atv02/data"

# realiza a leiura dos arquivos com os spams 'spam' e dos não spam 'ham'
data = DataFrame({'message': [], 'class': []})
data = data.append(dataFrameFromDirectory(path+'/spam', 'spam'))
data = data.append(dataFrameFromDirectory(path+'/ham', 'ham'))

Vamos exibir os primeiros registros do aquivo

In [25]:
# a função head exibe as primeiras linhas do arquivo
# podemos definir inclusive quantas linhas passando um parâmetro: head(10) exibe os 10 primeiros
data.head()

,class,message
/var/www/html/pos-graduacao/RDP/atv02/data/spam/00482.980c7ceb9333bc5027cd8d1d360a8c6f,spam,AFFORDABLE DOMAIN REGISTRATION FOR EVERYONE\n\...
/var/www/html/pos-graduacao/RDP/atv02/data/spam/00131.d955acc659fb151479460f9dd2f87efe,spam,==============================================...
/var/www/html/pos-graduacao/RDP/atv02/data/spam/00424.9acca894169b3162d76ebddb69097f3c,spam,\n\n<html>\n\n<head>\n\n\n\n</head>\n\n<body b...
/var/www/html/pos-graduacao/RDP/atv02/data/spam/00057.0a2e17bde9485e999ac2259df38528e2,spam,Lowest rates available for term life insurance...
/var/www/html/pos-graduacao/RDP/atv02/data/spam/00222.77293b7002c5749b9d31a99b2f4e0366,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."


Então vamos usar CountVectorizer para separar cada mensagem em uma lista de palavras e utilizá-las para treinar um classificador Naive Bayes. A função fit() realiza o treinamento

In [26]:
# cria o vetorizador de palavras da mensagem
vectorizer = CountVectorizer()
# conta a frequencia de cada palavra em cada mensagem
counts = vectorizer.fit_transform(data['message'].values)
# cria o classificador
classifier = MultinomialNB()
# define qual a saida correta para cada mensagem
targets = data['class'].values
# realiza o treinamento do classificador
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Vamos testar. Criamos um e-mail ficticio e verificamos se o classificador o identifica como spam ou não

In [27]:
#mensagem de exemplo
spam = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
notspam = ['Job Interview', "Hi Bob, are you free 10am tomorrow? We have an open hour and your test was rescheduled. Please dont be late"]
# conta a frequencia das palavras da mensagem
example_counts = vectorizer.transform(spam)
# realiza a predição
predictions = classifier.predict(example_counts)
predictions[0]

'spam'